In [2]:
# !pip install opensubtitlescom

In [7]:
# method to download and save imdb data to data/raw/
import requests

def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

url = 'https://datasets.imdbws.com/'
basics = 'title.basics.tsv.gz'
ratings = 'title.ratings.tsv.gz'
path = '../data/raw/'

# download_data(f'{url}{ratings}', f'{path}{ratings}')
# download_data(f'{url}{basics}', f'{path}{basics}')

In [8]:
import pandas as pd

# load the data
basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')
ratings_df = pd.read_csv(f'{path}{ratings}', sep='\t', na_values='\\N')

C:\Users\kyle0\AppData\Local\Temp\ipykernel_30140\1510432930.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')


In [9]:
# filter out only movies
movies_df = basics_df[basics_df['titleType'] == 'movie']

In [10]:
movies_df = movies_df.merge(ratings_df, on='tconst')

In [11]:
# sort by number of votes
movies_df = movies_df.sort_values('numVotes', ascending=False)

In [12]:
# remove adult movies
movies_df = movies_df[movies_df['isAdult'] == 0]

In [13]:
# make new column called imdb_id. This is the same as tconst but with 'tt' removed
movies_df['imdb_id'] = movies_df['tconst'].str[2:]

# now remove leading zeroes (for the subtitle api)
movies_df['imdb_id'] = movies_df['imdb_id'].str.lstrip('0')

In [14]:
df = movies_df[['tconst', 'imdb_id', 'primaryTitle', 'genres', 'startYear', 'averageRating', 'numVotes']]
df = df.reset_index(drop=True)

# drop startYear if it is NaN
df = df.dropna(subset=['startYear'])

df['startYear'] = df['startYear'].astype(int)
df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)

# save as tsv
df[:50000].to_csv('../data/processed/movies.tsv', sep='\t', index=False)

In [1]:
from opensubtitlescom import OpenSubtitles
import pandas as pd
import json
import os
from dotenv import load_dotenv
load_dotenv()  # Loads variables from the .env file

APP_NAME = os.getenv('APP_NAME')
API_KEY = os.getenv('API_KEY')
MY_USERNAME = os.getenv('MY_USERNAME')
MY_PASSWORD = os.getenv('MY_PASSWORD')

# Initialize the OpenSubtitles client
subtitles = OpenSubtitles(APP_NAME, API_KEY)
subtitles.login(MY_USERNAME, MY_PASSWORD)

df = pd.read_csv('../data/processed/movies.tsv', sep='\t')

In [2]:
# list movie ids
ids = df['imdb_id'][:20000].tolist()
# ids
# get title from df based on imdb_id
def get_title(imdb_id):
    return df[df['imdb_id'] == imdb_id]['primaryTitle'].values[0]


In [4]:
import time

subtitle_path = "../data/raw/subtitles/"
subs_list = os.listdir(subtitle_path)

# if id is in missing.txt, skip
with open(subtitle_path + 'missing.txt') as f:
    missing = f.read().splitlines()

for i in ids:
    if str(i) in missing:
        print(f'{i}, {get_title(i)} - subtitle missing')
        continue

    prefix = f'{subtitle_path}{i} - '  # Construct the base part of the filename

    sub_found = False

    # Check if subtitles already exist
    for file in os.listdir(subtitle_path):
        if file.startswith(f'{i} - '):
            sub_found = True
            break

    if sub_found:
        # print(f'Subtitles for {i} already exist')
        continue

    try:
        # Search for subtitles based on imdb_id
        # response = subtitles.search(imdb_id=i, languages="en", order_by="download_count", order_direction="desc")
        response = subtitles.search(imdb_id=i, languages="en")
        # time.sleep(1)

        # Convert the response to a Json format
        response_json = response.to_json()
        r = json.loads(response_json)
    except:
        print(f'Error searching for subtitles for {i} - {get_title(i)}')
        time.sleep(5)
        continue

    try:
        title = r['data'][0]['title'] # Get the title of the movie
    except:
        print(f'No subtitle found for {i} - {get_title(i)}')
        # append id to missing.txt
        with open(subtitle_path + 'missing.txt', 'a') as f:
            f.write(f'{i}\n')
        continue

    # Remove special characters but keep spaces
    title = ''.join(e for e in title if e.isalnum() or e.isspace())

    # Download and parse the subtitle object
    # try:
    #     srt = subtitles.download_and_parse(response.data[0])
    # except:
    #     print(f'Error downloading subtitle for {title}')
    #     srt = subtitles.download_and_parse(response.data[1])

    try:
        srt = subtitles.download_and_parse(response.data[0])
    except Exception as e1:  # Catch the initial error
        print(f'Error downloading first subtitle for {title}: {e1}')
        try:
            srt = subtitles.download_and_parse(response.data[1])
        except Exception as e2:
            print(f'Error downloading second subtitle for {title}: {e2}. Skipping...')
            continue  # Move on to the next item in the loop

    # Save
    with open(f'{prefix}{title}.txt', 'w', encoding='utf-8') as f:
        for i in srt:
            # remove '<i>' and '</i>' tags
            i.content = i.content.replace('<i>', '').replace('</i>', '')
            f.write(i.content + '\n')

9531772, Sooryavanshi - subtitle missing
5988370, Reis - subtitle missing
7825208, Marighella - subtitle missing
7668842, Enes Batur: Imagination or Reality? - subtitle missing
1725047, My Little Princess - subtitle missing
4884540, Angels Apocalypse - subtitle missing
85809, Koyaanisqatsi - subtitle missing
6038600, Smolensk - subtitle missing
7221896, Cumali Ceber: Allah Seni Alsin - subtitle missing
29808429, Color of Victory - subtitle missing
27719898, Kaiva - subtitle missing
3666024, The Red Turtle - subtitle missing
4465156, Unbreakable Souls - subtitle missing
1788979, 67th Street, New York, NY - subtitle missing
15145764, Freddy - subtitle missing
6990206, Crimea - subtitle missing
4330758, Black Hawk Down - subtitle missing
1754730, The Way to Paradise - subtitle missing
9851854, Major - subtitle missing
2183014, Love at First Sight - subtitle missing
3655326, Life Is a Dream - subtitle missing
3120430, Mantus - subtitle missing
4458206, Code Name: K.O.Z. - subtitle missing


API `subtitles?imdb_id=1365474&languages=en` failed with 502: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.opensubtitles.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb

Error searching for subtitles for 1365474 - Stricken
No subtitle found for 3500544 - Off Course
No subtitle found for 2573438 - G.D.O. BlackCat


API `subtitles?imdb_id=112767&languages=en` failed with 502: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.opensubtitles.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-

Error searching for subtitles for 112767 - Cyclo


...

Error searching for subtitles for 2113822 - Back to 1942
Error downloading first subtitle for The Wrecking Crew: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs)
Error downloading second subtitle for The Wrecking Crew: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs). Skipping...
Error downloading first subtitle for UFO: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs)
Error downloading second subtitle for UFO: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs). Skipping...
Error downloading first subtitle for Lucky Luke and the Daltons: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs)
Error downloading second subtitle for Lucky Luke and the Daltons: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs). Skipping...
Error downloading

API `subtitles?imdb_id=15234108&languages=en` failed with 502: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.opensubtitles.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:m

Error searching for subtitles for 15234108 - Networker Baire


API `subtitles?imdb_id=320042&languages=en` failed with 502: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.opensubtitles.com | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-

Error searching for subtitles for 320042 - Kamchatka
Error downloading first subtitle for Death of a Superhero: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs)
Error downloading second subtitle for Death of a Superhero: Download limit reached. Please upgrade your account or wait for your quota to reset (~24hrs). Skipping...
